# Table of Toronto Neighborhoods
All rows with Borough='Not assigned' were deleted

In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [126]:
Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [141]:
toronto_one= pd.get_dummies(Toronto[['Neighborhood']], prefix="", prefix_sep="")
toronto_one['Borough'] = Toronto['Borough'] 
fixed_columns = [toronto_one.columns[-1]] + list(toronto_one.columns[:-1])
toronto_one = toronto_one[fixed_columns]

In [143]:
toronto_one.shape

(103, 99)

In [167]:
toronto_grouped = toronto_one.groupby('Borough').mean().reset_index()
toronto_grouped.head()

,Borough,Agincourt,"Alderwood, Long Branch","Bathurst Manor, Wilson Heights, Downsview North",Bayview Village,"Bedford Park, Lawrence Manor East",Berczy Park,"Birch Cliff, Cliffside West","Brockton, Parkdale Village, Exhibition Place",Business reply mail Processing Centre,...,"West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale",Westmount,Weston,"Wexford, Maryvale",Willowdale,"Willowdale, Newtonbrook",Woburn,Woodbine Heights,York Mills West,"York Mills, Silver Hills"
0,Central Toronto,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Downtown Toronto,0.0,0.000000,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,East Toronto,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,East York,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
4,Etobicoke,0.0,0.083333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.083333,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
Toronto_grouped = Toronto.groupby('Borough').mean().reset_index()
Toronto_grouped.head()

,Cluster Labels,Borough,Latitude,Longitude
0,1,Central Toronto,43.701980,-79.398954
1,1,Downtown Toronto,43.654597,-79.383972
2,3,East Toronto,43.669436,-79.324654
3,4,East York,43.700303,-79.335851
4,1,Etobicoke,43.660043,-79.542074


I will use all the Boroughs and create 5 clusters

In [145]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Borough'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 3, 4, 1, 2, 1, 1, 1, 0], dtype=int32)

In [152]:
Toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_grouped

,Cluster Labels,Borough,Latitude,Longitude
0,1,Central Toronto,43.701980,-79.398954
1,1,Downtown Toronto,43.654597,-79.383972
2,3,East Toronto,43.669436,-79.324654
3,4,East York,43.700303,-79.335851
4,1,Etobicoke,43.660043,-79.542074
5,2,Mississauga,43.636966,-79.615819
6,1,North York,43.750727,-79.429338
7,1,Scarborough,43.766229,-79.249085
8,1,West Toronto,43.652653,-79.449290
9,0,York,43.690797,-79.472633


In [ ]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

In [166]:
map_clusters = folium.Map(location=[43.651070, 	-79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_grouped['Latitude'], Toronto_grouped['Longitude'], Toronto_grouped['Borough'], Toronto_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters